# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Limeq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Limeq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    token = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in token:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    clean_tokens = [tok for tok in clean_tokens if tok not in stopwords.words("english")]
      
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, binary=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

9min 35s ± 1.98 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
Y_pred = pipeline.predict(X_test)
Y_pred_df = pd.DataFrame(Y_pred, columns=Y_test.columns)

for col in Y_test.columns:
    print(col, classification_report(Y_test[col], Y_pred_df[col]))

related               precision    recall  f1-score   support

           0       0.71      0.34      0.46      1498
           1       0.82      0.96      0.89      5004
           2       0.57      0.15      0.24        52

    accuracy                           0.81      6554
   macro avg       0.70      0.48      0.53      6554
weighted avg       0.80      0.81      0.78      6554

request               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5420
           1       0.86      0.48      0.62      1134

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.89      0.90      0.89      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\Limeq\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


death               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6242
           1       0.83      0.12      0.22       312

    accuracy                           0.96      6554
   macro avg       0.89      0.56      0.60      6554
weighted avg       0.95      0.96      0.94      6554

other_aid               precision    recall  f1-score   support

           0       0.87      1.00      0.93      5685
           1       0.58      0.02      0.05       869

    accuracy                           0.87      6554
   macro avg       0.73      0.51      0.49      6554
weighted avg       0.83      0.87      0.81      6554

infrastructure_related               precision    recall  f1-score   support

           0       0.93      1.00      0.97      6116
           1       0.00      0.00      0.00       438

    accuracy                           0.93      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.87      0.93  

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001DE3FD84F28>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [47]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2, 3]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv
# {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50, 'tfidf__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
model = build_model()
model.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [60]:
print(model.best_params_)
Y_pred2 = model.predict(X_test)
Y_pred2_df = pd.DataFrame(Y_pred2, columns=Y_test.columns)

for col in Y_test.columns:
    print(col, classification_report(Y_test[col], Y_pred2_df[col]))

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50, 'tfidf__use_idf': True}
related               precision    recall  f1-score   support

           0       0.71      0.28      0.40      1498
           1       0.81      0.96      0.88      5004
           2       0.42      0.35      0.38        52

    accuracy                           0.80      6554
   macro avg       0.65      0.53      0.55      6554
weighted avg       0.79      0.80      0.77      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5420
           1       0.89      0.44      0.58      1134

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    a

floods               precision    recall  f1-score   support

           0       0.94      1.00      0.97      6002
           1       0.89      0.34      0.49       552

    accuracy                           0.94      6554
   macro avg       0.92      0.67      0.73      6554
weighted avg       0.94      0.94      0.93      6554

storm               precision    recall  f1-score   support

           0       0.95      0.99      0.97      5959
           1       0.78      0.46      0.58       595

    accuracy                           0.94      6554
   macro avg       0.86      0.72      0.77      6554
weighted avg       0.93      0.94      0.93      6554

fire               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6486
           1       1.00      0.01      0.03        68

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

e

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [82]:
def build_model2():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RidgeClassifierCV()))
    ])
    
    return pipeline

model2 = build_model2()
model2.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]),
                                                    

In [112]:
Y_pred3 = model2.predict(X_test)
Y_pred3_df = pd.DataFrame(Y_pred3, columns=Y_test.columns)

for col in Y_test.columns:
    print(col, classification_report(Y_test[col], Y_pred3_df[col]))
    

              precision    recall  f1-score   support

           0       0.70      0.48      0.57      1509
           1       0.85      0.94      0.89      5002
           2       0.58      0.26      0.35        43

    accuracy                           0.83      6554
   macro avg       0.71      0.56      0.61      6554
weighted avg       0.82      0.83      0.82      6554

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      5444
           1       0.86      0.54      0.66      1110

    accuracy                           0.91      6554
   macro avg       0.89      0.76      0.80      6554
weighted avg       0.90      0.91      0.90      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6475
           1       0.88      0.19      0.31        79

    accuracy                           0.99      6554
   macro avg       0.94      0.59      0.65      6554
weighted avg       0.99      0.99      0.99      6554

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5938
           1       0.90      0.69      0.78       616

    accuracy                           0.96      6554
   macro avg       0.93      0.84      0.88      6554
weighted avg       0.96      0.96      0.96      6554

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6424
           1       0.70      0.24      0.36       130

    accuracy                           0.98      6554
   macro avg       0.84      0.62      0.67      6554
weighted avg       0.98      0.98      0.98      6554

              prec

### 9. Export your model as a pickle file

In [118]:
pickle.dump(model2, open('message_model', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.